# TO RUN
1. Make sure you installed all the dependencies
2. open **'init_notebook.py'** and change the **projectFolder** to the src folder of the repository.
3. In the following cell change the dataDir to the data directory of the InD dataset

In [ ]:
# dataDir = "/Users/zarifikram/Documents/Repos/drone-dataset-tools/data/ind"
dataDir = "E:/Datasets/inD-dataset-v1.0/data/"
# dataDir = "G:AV datasets/inD-dataset-v1.0/data"

In [ ]:
exec(open('init_notebook.py').read())

In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
from loguru import logger
logger.remove()
logger.add(sys.stdout, format="{level} {message}", filter="my_module", level="INFO")
from joblib import Parallel, delayed
from extractors.loader import Loader
from extractors.LocationData import LocationData

In [ ]:
loader = Loader(dataDir, 'inD')
loader.locationToRecordingIds

# Step 1: Load all the location data, downsample them, extract SceneData and save location data


In [ ]:
# No Job Lib Version
# locations = [1, 2, 3, 4]
# locDataList = []

# for loc in locations:
#     locData = loader.getLocationData(loc, useSceneConfigToExtract=True, downSampleFps=10)
#     locData.save("../data") # whole thing as a dill object
    # locDataList.append(locData)

In [ ]:
# Job lib version
# locations = [1, 2, 3, 4]
# locDataList = Parallel(n_jobs=4)(
#         delayed(loader.getLocationData)(loc, useSceneConfigToExtract=True, downSampleFps=10) for loc in locations
# )
for locData in locDataList:
    locData.save("../data")

# Step 2: Clip Trajectories, find fast pedestrians, build local trajectories

In [ ]:
# we can also load loc data using this: 
# loc2dataFromFile = LocationData.load("../data/location-2", "2023-03-25-fps-5-all.dill")
# No Job Lib Version
for locDataFromFile in locDataList:
    locDataFromFile.buildLocalInformationForScenes()
    

In [ ]:
# # Job lib version
# _ = Parallel(n_jobs=4)(
#         delayed(locData.buildLocalInformationForScenes)() for locData in locDataList
# )
# for locData in locDataList:
#     locData.saveCrossingDf("../data")
#     locData.saveSceneDataOnly("../data")
#     locData.save("../data") # whole thing as a dill object

In [32]:
for locData in locDataList:
    locData.saveCrossingDf("../data")
    # locData.saveSceneDataOnly("../data")
    # locData.save("../data") # whole thing as a dill object

1
2
3
4


## Step 3: Analyze problem tracks

In [42]:
from tools.TrajectoryUtils import TrajectoryUtils
from tools.TrajectoryVisualizer import TrajectoryVisualizer
import matplotlib.pyplot as plt
import numpy as np
visualizer = TrajectoryVisualizer(loader)

In [ ]:
for locData in locDataList:
    sceneIds = locData.getSceneIds()
    # print(sceneIds)
    if locData.locationId == 2:
        sceneIds.remove('140') # somethig is fishy
        
    # print(sceneIds)
    for sceneId in sceneIds:
        
        sceneData = locData.getSceneData(sceneId) # loads precomputed
#         print("scene id", sceneId)
#         print(sceneData.warnings)
#         print(sceneData.problematicIds)
        visualizer.showSceneProblems(sceneData)

In [ ]:
pedDf.describe()

In [ ]:
loc2Data = locDataList[1]
loc2Data.locationId

In [ ]:
sceneData= loc2Data.getSceneData(240)
sceneData.buildLocalInformation(loc2Data.transformer, loc2Data.cleaner, force=True)
# pedDf = scene210.getPedDataInSceneCoordinates()
# pedDf.describe()

INFO:root:SceneData 240: moving outlier peds to others. We should only find outliers in the clipped trajectories?
INFO:root:Scene 240: clipping original data


using range (0.0, 3.0)


clipping ped trajectories for scene # 240 with width offset 4.0: 100%|███████████████| 185/185 [00:13<00:00, 13.70it/s]
clipping other trajectories for scene # 240:  99%|████████████████████████████████▊| 1160/1167 [01:10<00:00, 11.62it/s]

In [58]:
sceneData.problematicIds

defaultdict(<function extractors.SceneData.SceneData.buildLocalInformation.<locals>.<lambda>()>,
            {'fast_pedestrian': {19026,
              19039,
              19040,
              19049,
              19063,
              19288,
              20295,
              21225,
              22227,
              23300,
              25420,
              25471,
              26123,
              26323,
              27252,
              27363,
              27572,
              29238,
              29240,
              29285,
              29513,
              29575}})

In [54]:
loc2Data.cleaner.minSpeed

0.0

In [55]:
loc2Data.cleaner.maxSpeed

3.0